In [1]:
%load_ext autoreload
%autoreload 2
%cd /home/florian/GitRepos/activeCell-ACDC
!ls

import json
import random as rd
import matplotlib.image as mpimg
import cv2
import wandb
import torch
import os
import logging
import pandas as pd
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import time

from detectron2.utils.visualizer import Visualizer
from detectron2.utils.logger import setup_logger
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer

from src.globals import *
from utils.visualization.show_image import show_image, plot_prediction
from utils.register_datasets import register_datasets, get_dataset_name
from utils.config_builder import get_config
from utils.notebook_utils import *
from utils.evaluation import SingleImageCOCOEvaluator
from utils.evaluation_accuracy import AccuracyEvaluator
from utils.evaluation_uncertainty import evaluate_uncertainties


from src.test import do_test
from src.active_learning.al_trainer import *
from src.active_learning.mc_dropout_sampler import *
from src.active_learning.tta_sampler import *


logger = setup_logger(output="./log/main.log",name="null_logger") 
logger.addHandler(logging.NullHandler())
logging.getLogger('detectron2').setLevel(logging.WARNING)
logging.getLogger('detectron2').addHandler(logging.NullHandler())

/home/florian/GitRepos/activeCell-ACDC
ac_acdc_env					output
acdc_large_cls_test_slim_coco_format.json	pipeline_configs
acdc_large_cls_test_slim_coco_format.json.lock	README.md
al_output					requirements.txt
data						shell_scripts
experiments					src
local						utils
log						wandb
notebooks


In [2]:
dataset = ACDC_LARGE_CLS
test_dataset_name = dataset + "_test_slim"
config_name = "final_random_al"

model_path = "/home/florian/GitRepos/activeCell-ACDC/output"

In [3]:
register_datasets()
train_data = DatasetCatalog.get(get_dataset_name(
    dataset, DATASETS_DSPLITS[dataset][0]
))

test_data = DatasetCatalog.get(test_dataset_name)

registering acdc_large_cls dataset
registering chlamy_dataset dataset


In [4]:
wandb.init(
    project="activeCell-ACDC",
    name="",
    sync_tensorboard=True,
    mode="disabled",
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [5]:
default_cfg = get_config(config_name)
cfg = default_cfg
cfg.OUTPUT_DIR = "./al_output/classes_acdc_large_al"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05#.4
#cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.2
#cfg.TEST.DETECTIONS_PER_IMAGE = 50

In [6]:
model = load_model(cfg, os.path.join(model_path, "best_model.pth"))
#model = load_model(cfg, os.path.join(model_path, "last_model1200.pth"))


In [13]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
stragtegy = MCDropoutSampler(cfg)
model = patch_module(model)
start_time = time.time()
res_unc = evaluate_uncertainties(cfg, test_dataset_name, model, stragtegy)
print("--- %s seconds ---" % (time.time() - start_time))

/home/florian/GitRepos/activeCell-ACDC/ac_acdc_env/lib/python3.10/site-packages/baal/bayesian/common.py:62: UserWarning: No layer was modified by patch_module!
  warnings.warn("No layer was modified by patch_module!", UserWarning)


registering acdc_large_cls dataset
registering chlamy_dataset dataset


100%|██████████| 509/509 [15:23<00:00,  1.81s/it]

--- 926.7615103721619 seconds ---


In [7]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
stragtegy = TTASampler(cfg)
model = load_model(cfg, os.path.join(model_path, "best_model.pth"))
start_time = time.time()
res_unc = evaluate_uncertainties(cfg, test_dataset_name, model, stragtegy)
print("--- %s seconds ---" % (time.time() - start_time))

registering acdc_large_cls dataset
registering chlamy_dataset dataset


  0%|          | 0/509 [00:00<?, ?it/s]/home/florian/GitRepos/activeCell-ACDC/ac_acdc_env/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  5%|▍         | 23/509 [00:55<19:42,  2.43s/it]

In [7]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
cfg.AL.NUM_MC_SAMPLES = 1
stragtegy = MCDropoutSampler(cfg)
model = load_model(cfg, os.path.join(model_path, "best_model.pth"))
start_time = time.time()

# load dataset
register_datasets()
data_set = DatasetCatalog.get(test_dataset_name)
for im_json in tqdm(data_set):
    im = stragtegy.load_image(im_json)
    stragtegy.get_samples(model, im, 1)
    
print("--- %s seconds ---" % (time.time() - start_time))

registering acdc_large_cls dataset
registering chlamy_dataset dataset


  0%|          | 0/509 [00:00<?, ?it/s]/home/florian/GitRepos/activeCell-ACDC/ac_acdc_env/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 509/509 [00:42<00:00, 11.84it/s]

--- 46.338428020477295 seconds ---
